In [1]:
import re
import json
import pandas as pd
from utils import to_snake_case, load_json, dump_json

# Upload dataset and convert to json

In [2]:
def prepare_df(pth: str):
    
    df = pd.read_excel(pth)

    cols = df.columns.to_list()
    cols[-1] = "MAXIMUM RETAIL PRICE"
    df.columns = cols

    df = df.rename(columns=to_snake_case)

    df.dropna(subset=['maximum_retail_price'], inplace=True)
    df.dropna(subset=['active_substance'], inplace=True)
    df.dropna(subset=['name_of_medicinal_product'], inplace=True)
    df.reset_index(inplace=True, drop=True)
    df['name'] = df['name_of_medicinal_product'].apply(lambda x: x.split()[0])
    df['name'] = df['name'].apply(lambda x: x.split('/')[0])
    df['name'] = df['name'].apply(lambda x: x.split('+')[0])
    gr_exception = "Εξαίρεση αναγραφής τιμής στον τιμοκατάλογο (Κανονισμός 4, ΚΔΠ 98/2019)"
    gr_exception_1 = "* Εξαίρεση αναγραφής τιμής στον τιμοκατάλογο (Κανονισμός 4, ΚΔΠ 98/2019)"
    df['maximum_retail_price'].replace(gr_exception, 'not_listed', inplace=True)
    df['maximum_retail_price'].replace(gr_exception_1, 'not_listed', inplace=True)

    new_order = [
    'name', 'maximum_retail_price', 'active_substance', 
    'name_of_medicinal_product', 'package', 
    'marketing_authorisation_holder', 'local_pricing_representative',
    'pricing_code'
    ]

    return df[new_order]

In [40]:
df_2023 = prepare_df("csv/ΤΙΜΟΚΑΤΑΛΟΓΟΣ ΣΤ2023_24-11-2023_(05-12-2023)_WEB_EN.xlsx")
df_2024 = prepare_df("csv/ΤΙΜΟΚΑΤΑΛΟΓΟΣ Β2024_10.04.2024_(29.03.2024)_WEB_EN.xlsx")

In [33]:
df_2023.reset_index().to_json('drugs_2023.json', orient='records', indent=4)

In [34]:
df_2024.reset_index().to_json('drugs_2024.json', orient='records', indent=4)

In [3]:
drugs = load_json('drugs.json')

# load jsons

In [58]:
df_2023 = load_json('drugs_2023.json')
df_2024 = load_json("drugs_2024.json")

In [73]:
DATE1 = '29-03-2024'
DATE2 = '05-12-2023'

def fill_dc(ind: int, d: dict) -> dict:
    
    dc = {}
    dc['ind'] = ind
    dc['_name'] = d['name'].capitalize()
    dc['_full_name'] = d['name_of_medicinal_product'].capitalize()
    dc['_active'] = d['active_substance'].capitalize()
    dc['_package'] = d['package'].split(' ', 1)
    dc['_price'] = d['maximum_retail_price']

    dc[DATE1] = {}
    dc[DATE2] = {}

    return dc

In [82]:
res = []
ind = 0

for d3 in df_2023:

    dc = fill_dc(ind, d3)

    dc[DATE2].update(d3)

    res.append(dc)
    ind += 1

In [83]:
res2 = []
for d4 in df_2024:
    code = d4.get('pricing_code')

    new = True
    for r in res:
        if code == r[DATE2]['pricing_code']:
            r[DATE1].update(d4)
            new = False
            break


    if new:
        dc = fill_dc(ind, d4)
        dc[DATE1].update(d4)
        
        res2.append(dc)
        ind += 1     


In [88]:
res += res2
sorted_res = sorted(res, key=lambda x: x['_name'])

for i, s in enumerate(sorted_res):
    s['ind'] = i
    if s[DATE1] != {}:
        s['_price'] = s[DATE1]["maximum_retail_price"]
    else:
        s['_price'] = s[DATE2]["maximum_retail_price"]


# dump_json("drugs.json", sorted_res)

# Clear drugs.json

In [3]:
df = load_json("drugs.json")

In [9]:
def format_active(string: str, sign: str) -> list:
    return [s.strip().capitalize() for s in string.replace(sign, ",").split(',') if s != '']

In [10]:
ind = 1
for d in df:
    active = d.get('_active')
    for a in active:
        sign = '\n'
        if sign in a and len(active) == 1:
            d['_active'] = format_active(a, sign)
            print(ind, d['ind'], len(active))
            ind += 1
            

1 145 1
2 146 1
3 2108 1
4 3533 1
5 3950 1
6 4475 1
7 5522 1
8 5523 1
9 5593 1
10 5671 1
11 5856 1
12 5857 1
13 5858 1
14 5859 1
15 5860 1
16 5861 1
17 5862 1
18 5863 1


In [11]:
df[145]

{'ind': 145,
 '_name': 'Akeega',
 '_full_name': 'Akeega tablet, film coated 100mg/500mg',
 '_active': ['Niraparib', 'Abiraterone acetate'],
 '_package': ['PACK', 'with 56 tabs in blister(s)'],
 '_price': 5104.04,
 '29-03-2024': {'index': 143,
  'name': 'AKEEGA',
  'maximum_retail_price': 5104.04,
  'active_substance': 'NIRAPARIB\nABIRATERONE ACETATE',
  'name_of_medicinal_product': 'AKEEGA TABLET, FILM COATED 100MG/500MG',
  'package': 'PACK WITH 56 TABS IN BLISTER(S)',
  'marketing_authorisation_holder': 'JANSSEN-CILAG INTERNATIONAL NV',
  'local_pricing_representative': 'VARNAVAS HADJIPANAYIS LTD',
  'pricing_code': 'C172202/1'},
 '05-12-2023': {}}

In [12]:
# dump_json('drugs.json', df)

Ambisome
AREGALU
CEFUROXIME 1076
eye drops
Tavneos

# Prepare active

In [24]:
active: list = load_json('active.json')
drugs: list = load_json('drugs.json')

In [26]:
for d in drugs:
    name = d.get('_name').upper()

    exist = False
    for a in active:
        if name == a.get("base_name"):
            exist = True
            break

    if not exist:
        dc = {
            "ind": 180,
            "base_name": name.upper(),
            "drugs": [name],
            "active": d.get('_active')
            }
        
        active.append(dc)
        

In [30]:
sorted_active = sorted(active, key=lambda x: x['base_name'])

In [34]:
ind = 0
for s in sorted_active:
    s['ind'] = ind
    ind += 1

In [36]:
# dump_json('active.json', sorted_active)